# Trello SQL Challenge

### Sunne Kuo 11/19/2016

Given the data file linked below, what is the name of the tag with the lowest id that has a post count equal to the median post count?

Please include the tag name in the field below, along with the SQL code you wrote to get the answer.

Data file*:  http://bit.ly/2cXJNe2

* Data saved from https://archive.org/details/stackexchange. Live data may be different than the provided snapshot.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Import and read csv data
tags_csv = pd.read_csv('http://bit.ly/2cXJNe2')

In [3]:
#Get column names
column_names = tags_csv.columns[0].split('~')
column_names[0] = 'ID'
column_names[2] = 'Cnt_hits'

In [4]:
#Split the data into a list and remove the original column
tags_csv['list_of_data'] = tags_csv.ix[:,0].apply(lambda x: x.split('~'))
tags_csv.drop(tags_csv.columns[0], axis=1, inplace=True)

In [5]:
#Create dataframe from lists contained in rows
tags = pd.DataFrame({column_names[0]: tags_csv['list_of_data'].apply(lambda x: x[0]),
                     column_names[1]: tags_csv['list_of_data'].apply(lambda x: x[1]),
                     column_names[2]: tags_csv['list_of_data'].apply(lambda x: x[2]),
                     column_names[3]: tags_csv['list_of_data'].apply(
                                                    lambda x: x[3] if len(x)>3 else np.nan),
                     column_names[4]: tags_csv['list_of_data'].apply(
                                                    lambda x: x[4] if len(x)>3 else np.nan)},
                     columns = column_names)

In [6]:
#Convert Count column into integer type
tags['Cnt_hits'] = tags['Cnt_hits'].apply(lambda x: int(x))

In [7]:
#Select Tag Name with the lowest id that has a post count equal to the median post count using Pandas indexing
answer = tags.ix[tags['Cnt_hits'] == int(tags['Cnt_hits'].median()), 
                 ['ID', 'TagName']].sort_values('ID').reset_index().ix[0,'TagName']

In [8]:
print("The tag name with the lowest ID whose post count = median post count is: {}"
      .format(answer))

The tag name with the lowest ID whose post count = median post count is: chickens


## SQL Query

In [9]:
#Run on sqlite
from pandasql import *
pysqldf = lambda q: sqldf(q, globals())

In [10]:
q2  = """

SELECT TagName
FROM tags
GROUP BY ID
HAVING Cnt_hits = 
(SELECT CAST(AVG(Cnt_hits) AS INTEGER) AS Count_median
 FROM (SELECT Cnt_hits
      FROM tags
      ORDER BY Cnt_hits
      LIMIT 2 - (SELECT COUNT(*) FROM tags) % 2    
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM tags)))
LIMIT 1

"""

pysqldf(q2)

,TagName
0,chickens
